<p style="text-align:center">
    <a href="https://skills.network/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDS0321ENSkillsNetwork26802033-2022-01-01" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Hands-on Lab: Interactive Visual Analytics with Folium**


Estimated time needed: **40** minutes


The launch success rate may depend on many factors such as payload mass, orbit type, and so on. It may also depend on the location and proximities of a launch site, i.e., the initial position of rocket trajectories. Finding an optimal location for building a launch site certainly involves many factors and hopefully we could discover some of the factors by analyzing the existing launch site locations.


In the previous exploratory data analysis labs, you have visualized the SpaceX launch dataset using `matplotlib` and `seaborn` and discovered some preliminary correlations between the launch site and success rates. In this lab, you will be performing more interactive visual analytics using `Folium`.


## Objectives


This lab contains the following tasks:

*   **TASK 1:** Mark all launch sites on a map
*   **TASK 2:** Mark the success/failed launches for each site on the map
*   **TASK 3:** Calculate the distances between a launch site to its proximities

After completed the above tasks, you should be able to find some geographical patterns about launch sites.


Let's first import required Python packages for this lab:


In [ ]:
import piplite
await piplite.install(['folium'])
await piplite.install(['pandas'])

In [ ]:
import folium
import pandas as pd

In [ ]:
# Import folium MarkerCluster plugin
from folium.plugins import MarkerCluster
# Import folium MousePosition plugin
from folium.plugins import MousePosition
# Import folium DivIcon plugin
from folium.features import DivIcon

If you need to refresh your memory about folium, you may download and refer to this previous folium lab:


[Generating Maps with Python](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DV0101EN-SkillsNetwork/labs/v4/DV0101EN-Exercise-Generating-Maps-in-Python.ipynb)


In [ ]:
import folium
import pandas as pd
from folium.plugins import MarkerCluster, MousePosition

# Sample structure of your launch site DataFrame
# Replace this with your actual dataset if needed
df = pd.DataFrame({
    'LaunchSite': ['CCAFS LC-40', 'VAFB SLC-4E', 'KSC LC-39A', 'CCAFS SLC-41'],
    'Latitude': [28.5623, 34.6321, 28.5735, 28.5632],
    'Longitude': [-80.5774, -120.6108, -80.6469, -80.5768]
})

# Initialize base map around central U.S. region
site_map = folium.Map(location=[28.5, -96.0], zoom_start=4)

# Add a marker cluster to group nearby sites
marker_cluster = MarkerCluster().add_to(site_map)

# Plot each launch site on the map
for _, row in df.iterrows():
    folium.Marker(
        location=[row['Latitude'], row['Longitude']],
        popup=row['LaunchSite'],
        icon=folium.Icon(color='blue', icon='rocket', prefix='fa')
    ).add_to(marker_cluster)

# Add mouse position plugin to show coordinates
MousePosition().add_to(site_map)

# Display the map
site_map


First, let's try to add each site's location on a map using site's latitude and longitude coordinates


The following dataset with the name `spacex_launch_geo.csv` is an augmented dataset with latitude and longitude added for each site.


In [ ]:
# Download and read the `spacex_launch_geo.csv`
from js import fetch
import io

URL = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/datasets/spacex_launch_geo.csv'
resp = await fetch(URL)
spacex_csv_file = io.BytesIO((await resp.arrayBuffer()).to_py())
spacex_df=pd.read_csv(spacex_csv_file)

Now, you can take a look at what are the coordinates for each site.


In [ ]:
# Select relevant sub-columns: `Launch Site`, `Lat(Latitude)`, `Long(Longitude)`, `class`
spacex_df = spacex_df[['Launch Site', 'Lat', 'Long', 'class']]
launch_sites_df = spacex_df.groupby(['Launch Site'], as_index=False).first()
launch_sites_df = launch_sites_df[['Launch Site', 'Lat', 'Long']]
launch_sites_df

Above coordinates are just plain numbers that can not give you any intuitive insights about where are those launch sites. If you are very good at geography, you can interpret those numbers directly in your mind. If not, that's fine too. Let's visualize those locations by pinning them on a map.


We first need to create a folium `Map` object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.


In [ ]:
# Start location is NASA Johnson Space Center

nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=10)

We could use `folium.Circle` to add a highlighted circle area with a text label on a specific coordinate. For example,


In [ ]:
# Create a blue circle at NASA Johnson Space Center's coordinate with a popup label showing its name
circle = folium.Circle(nasa_coordinate, radius=1000, color='#d35400', fill=True).add_child(folium.Popup('NASA Johnson Space Center'))
# Create a blue circle at NASA Johnson Space Center's coordinate with a icon showing its name
marker = folium.map.Marker(
    nasa_coordinate,
    # Create an icon as a text label
    icon=DivIcon(
        icon_size=(20,20),
        icon_anchor=(0,0),
        html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'NASA JSC',
        )
    )
site_map.add_child(circle)
site_map.add_child(marker)

and you should find a small yellow circle near the city of Houston and you can zoom-in to see a larger circle.


Now, let's add a circle for each launch site in data frame `launch_sites`


*TODO:*  Create and add `folium.Circle` and `folium.Marker` for each launch site on the site map


An example of folium.Circle:


`folium.Circle(coordinate, radius=1000, color='#000000', fill=True).add_child(folium.Popup(...))`


An example of folium.Marker:


`folium.map.Marker(coordinate, icon=DivIcon(icon_size=(20,20),icon_anchor=(0,0), html='<div style="font-size: 12; color:#d35400;"><b>%s</b></div>' % 'label', ))`


In [ ]:
import folium
from folium.features import DivIcon

# Initialize the map centered on NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch site and add a circle and label marker
for _, row in launch_sites_df.iterrows():
    site_name = row['Launch Site']
    site_lat = row['Lat']
    site_long = row['Long']
    coordinate = [site_lat, site_long]

    # 🔵 Add a Circle marker for launch site
    folium.Circle(
        location=coordinate,
        radius=1000,
        color='#000000',
        fill=True,
        fill_opacity=0.6,
        popup=folium.Popup(site_name, parse_html=True)
    ).add_to(site_map)

    # 📍 Add a text label using DivIcon
    folium.Marker(
        location=coordinate,
        icon=DivIcon(
            icon_size=(150, 36),
            icon_anchor=(0, 0),
            html=f'<div style="font-size:12px; color:#d35400;"><b>{site_name}</b></div>'
        )
    ).add_to(site_map)

# Display the map
site_map


The generated map with marked launch sites should look similar to the following:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_markers.png">
</center>


Now, you can explore the map by zoom-in/out the marked areas
, and try to answer the following questions:

*   Are all launch sites in proximity to the Equator line?
*   Are all launch sites in very close proximity to the coast?

Also please try to explain your findings.


In [ ]:
import folium

# Initialize the map centered at NASA Johnson Space Center
nasa_coordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location=nasa_coordinate, zoom_start=5)

# Loop through each launch event in the dataset
for _, row in spacex_df.iterrows():
    launch_coord = [row['Lat'], row['Long']]
    site_name = row['Launch Site']
    outcome = row['class']  # 1 = success, 0 = failure

    # ✅ Set marker color based on launch outcome
    marker_color = 'green' if outcome == 1 else 'red'
    outcome_label = 'Success' if outcome == 1 else 'Failure'

    # 📍 Add a marker for the launch
    folium.Marker(
        location=launch_coord,
        popup=f"{site_name} — {outcome_label}",
        icon=folium.Icon(color=marker_color, icon='rocket', prefix='fa')
    ).add_to(site_map)

# 🗺️ Display the interactive map
site_map


Next, let's try to enhance the map by adding the launch outcomes for each site, and see which sites have high success rates.
Recall that data frame spacex_df has detailed launch records, and the `class` column indicates if this launch was successful or not


In [ ]:
spacex_df.tail(10)

Next, let's create markers for all launch records.
If a launch was successful `(class=1)`, then we use a green marker and if a launch was failed, we use a red marker `(class=0)`


Note that a launch only happens in one of the four launch sites, which means many launch records will have the exact same coordinate. Marker clusters can be a good way to simplify a map containing many markers having the same coordinate.


Let's first create a `MarkerCluster` object


In [ ]:
marker_cluster = MarkerCluster()


*TODO:* Create a new column in `spacex_df` dataframe called `marker_color` to store the marker colors based on the `class` value


In [ ]:

# Apply a function to check the value of `class` column
# If class=1, marker_color value will be green
# If class=0, marker_color value will be red
# Apply conditional logic to assign marker colors
spacex_df['marker_color'] = spacex_df['class'].apply(lambda x: 'green' if x == 1 else 'red')


*TODO:* For each launch result in `spacex_df` data frame, add a `folium.Marker` to `marker_cluster`


In [ ]:
import folium
from folium.plugins import MarkerCluster

# Create a marker cluster object
marker_cluster = MarkerCluster()

# Add the marker cluster to the map
site_map.add_child(marker_cluster)

# Add markers for each launch event
for _, record in spacex_df.iterrows():
    launch_location = [record['Lat'], record['Long']]
    marker = folium.Marker(
        location=launch_location,
        popup=f"{record['Launch Site']} — {'Success' if record['class'] == 1 else 'Failure'}",
        icon=folium.Icon(color='white', icon_color=record['marker_color'], icon='rocket', prefix='fa')
    )
    marker_cluster.add_child(marker)

# Display the updated map
site_map


Your updated map may look like the following screenshots:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster.png">
</center>


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_cluster_zoomed.png">
</center>


From the color-labeled markers in marker clusters, you should be able to easily identify which launch sites have relatively high success rates.


In [ ]:
# TASK 3: Calculate the distances between a launch site to its proximities
from geopy.distance import geodesic

# Function to calculate distances
def calculate_distances(launch_sites_df, spacex_df):
    distances = []
    for _, launch in spacex_df.iterrows():
        site_row = launch_sites_df[launch_sites_df['Launch Site'] == launch['Launch Site']].iloc[0]
        site_coord = (site_row['Lat'], site_row['Long'])
        launch_coord = (launch['Lat'], launch['Long'])
        distance_km = geodesic(site_coord, launch_coord).km
        distances.append(distance_km)
    return distances

# Apply to create a new column
spacex_df['Distance (km)'] = calculate_distances(launch_sites_df, spacex_df)



Next, we need to explore and analyze the proximities of launch sites.


Let's first add a `MousePosition` on the map to get coordinate for a mouse over a point on the map. As such, while you are exploring the map, you can easily find the coordinates of any points of interests (such as railway)


In [ ]:
from folium.plugins import MousePosition

# Define coordinate formatting function
formatter = "function(num) {return L.Util.formatNum(num, 5);};"

# Create MousePosition plugin to show Lat/Long on hover
mouse_position = MousePosition(
    position='topright',
    separator=' Long: ',
    empty_string='NaN',
    lng_first=False,
    num_digits=20,
    prefix='Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

# Add MousePosition to site map
site_map.add_child(mouse_position)

# Display the map with interactive coordinate tracking
site_map


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


Now zoom in to a launch site and explore its proximity to see if you can easily find any railway, highway, coastline, etc. Move your mouse to these points and mark down their coordinates (shown on the top-left) in order to the distance to the launch site.


In [ ]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

*TODO:* Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.


In [ ]:
# find coordinate of the closet coastline
# e.g.,: Lat: 28.56367  Lon: -80.57163
# distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
# Example: Coordinates of a coastline point marked using MousePosition on the map
coastline_coord = (28.5635, -80.5678)  # Replace with actual values from MousePosition

# Reference: Let's say we're checking from CCAFS LC-40
launch_site_row = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coord = (launch_site_row['Lat'], launch_site_row['Long'])

# 🧮 Calculate distance from launch site to coastline
distance_to_coast = calculate_distance(
    launch_coord[0], launch_coord[1],
    coastline_coord[0], coastline_coord[1]
)

print(f"Distance from CCAFS LC-40 to coastline: {distance_to_coast:.2f} km")


In [ ]:
import folium
from folium.features import DivIcon

# 🧭 Selected coastline coordinate (from MousePosition)
coastline_coord = (28.5635, -80.5678)  # Replace with your actual point

# 🚀 Reference launch site (e.g., CCAFS LC-40)
launch_site = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coord = (launch_site['Lat'], launch_site['Long'])

# 📏 Calculate distance
distance = calculate_distance(
    launch_coord[0], launch_coord[1],
    coastline_coord[0], coastline_coord[1]
)

# 📍 Add a distance marker at coastline point
distance_marker = folium.Marker(
    location=coastline_coord,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#d35400;"><b>{distance:.2f} KM</b></div>'
    )
)

# 🔧 Add marker to map
site_map.add_child(distance_marker)

# 🗺️ Show updated map
site_map


*TODO:* Draw a `PolyLine` between a launch site to the selected coastline point


In [ ]:
# Define coordinates for PolyLine: from launch site to selected coastline point
line_coords = [launch_coord, coastline_coord]  # tuple pairs like (lat, lon)

# Create PolyLine object
lines = folium.PolyLine(locations=line_coords, weight=2, color='blue')

# Add the line to the site map
site_map.add_child(lines)

# Display updated map
site_map


Your updated map with distance line should look like the following screenshot:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/launch_site_marker_distance.png">
</center>


*TODO:* Similarly, you can draw a line betwee a launch site to its closest city, railway, highway, etc. You need to use `MousePosition` to find the their coordinates on the map first


A railway map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/railway.png">
</center>


A highway map symbol may look like this:


In [ ]:
<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/highway.png">
</center>


A city map symbol may look like this:


<center>
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_3/images/city.png">
</center>


In [ ]:
from folium.features import DivIcon

# 🏙️ Example coordinates from MousePosition for a nearby city/railway/highway
poi_coord = (28.6142, -80.6943)  # Replace with actual coordinates from map exploration

# 🚀 Choose launch site (e.g., CCAFS LC-40)
launch_row = launch_sites_df[launch_sites_df['Launch Site'] == 'CCAFS LC-40'].iloc[0]
launch_coord = (launch_row['Lat'], launch_row['Long'])

# 📏 Calculate distance using haversine formula
poi_distance = calculate_distance(
    launch_coord[0], launch_coord[1],
    poi_coord[0], poi_coord[1]
)

# 🏷️ Add a labeled marker at the POI with distance
poi_marker = folium.Marker(
    location=poi_coord,
    icon=DivIcon(
        icon_size=(20, 20),
        icon_anchor=(0, 0),
        html=f'<div style="font-size: 12px; color:#0055aa;"><b>{poi_distance:.2f} KM</b></div>'
    )
)
site_map.add_child(poi_marker)

# 🔗 Add a line from the launch site to the POI
line = folium.PolyLine(locations=[launch_coord, poi_coord], weight=2, color='purple')
site_map.add_child(line)

# 🗺️ Display map
from IPython.display import display
display(site_map)
site_map



After you plot distance lines to the proximities, you can answer the following questions easily:

*   Are launch sites in close proximity to railways?
*   Are launch sites in close proximity to highways?
*   Are launch sites in close proximity to coastline?
*   Do launch sites keep certain distance away from cities?

Also please try to explain your findings.


# Next Steps:

Now you have discovered many interesting insights related to the launch sites' location using folium, in a very interactive way. Next, you will need to build a dashboard using Ploty Dash on detailed launch records.


## Authors


[Pratiksha Verma](https://www.linkedin.com/in/pratiksha-verma-6487561b1/)


<!--## Change Log--!>


<!--| Date (YYYY-MM-DD) | Version | Changed By      | Change Description      |
| ----------------- | ------- | -------------   | ----------------------- |
| 2022-11-09        | 1.0     | Pratiksha Verma | Converted initial version to Jupyterlite|--!>


### <h3 align="center"> IBM Corporation 2022. All rights reserved. <h3/>
